# Import

In [1]:
import numpy as np
import nibabel as nib
import csv
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
from tensorflow.keras.optimizers.legacy import Adam

# Data

In [2]:
from google.colab import drive
drive.mount('content')
import os
print('reached here')
print(os.listdir('./content/MyDrive/Me/Images'))
os.chdir('./content/MyDrive/Me/Images')
print(os.listdir('.'))
#!cd './drive/Me/Images'

Mounted at content
reached here
['ds003548']
['ds003548']


In [3]:
fmri_data = {}

In [4]:
for i in range(1,17):
    print('loading person', i)
    padi = str(i).rjust(2,'0')
    fmri_data['sub_'+str(i)] = {}
    for run in range(1,6):
        img = nib.load(f'./ds003548/sub-{padi}/func/sub-{padi}_task-emotionalfaces_run-{run}_bold.nii.gz')
        img_arr = np.array(img.dataobj)
        fmri_data['sub_'+str(i)][str(run)] = img_arr

loading person 1
loading person 2
loading person 3
loading person 4
loading person 5
loading person 6
loading person 7
loading person 8
loading person 9
loading person 10
loading person 11
loading person 12
loading person 13
loading person 14
loading person 15
loading person 16


In [5]:
#Shape of data is x,y,z,t
fmri_data['sub_1']['5'].shape

(64, 64, 35, 185)

In [6]:
inputs = {}

In [7]:
import csv
for i in range(1,6):
    print('loading run', i)
    with open(f"./ds003548/task-emotionalfaces_run-{i}_events.tsv") as file:
        tsv_file = csv.reader(file, delimiter="\t")
        inputs['run_'+str(i)] = [x for x in tsv_file]

loading run 1
loading run 2
loading run 3
loading run 4
loading run 5


In [8]:
inputs['run_2']

[['onset', 'duration', 'trial_type'],
 ['0', '30', 'sad'],
 ['30', '30', 'neutral'],
 ['60', '30', 'blank'],
 ['90', '30', 'angry'],
 ['120', '30', 'scrambled'],
 ['150', '30', 'happy'],
 ['180', '30', 'sad'],
 ['210', '30', 'neutral'],
 ['240', '30', 'blank'],
 ['270', '30', 'angry'],
 ['300', '30', 'scrambled'],
 ['330', '30', 'happy'],
 ['360', '10', 'end'],
 []]


"After their presentation, participants had to wait for 10 seconds before concluding the sequence, in order to capture the hemodynamic response (HR) elicited by the last stimuli."
Turn end into previous emotion?

In [9]:
#####   CHANGE END TO MOST RECENT EMOTION
print("Last trial type:",inputs['run_2'][-2][-1])
print("2nd last trial type:", inputs['run_2'][-3][-1])

for runs in inputs:  #run_1, run_2, run_3, run_4, run_5
  inputs[runs][-2][-1] = inputs[runs][-3][-1]

#check
for runs in inputs:
  print(inputs[runs][-2][-1])


Last trial type: end
2nd last trial type: happy
blank
happy
sad
scrambled
blank


In [10]:
np.repeat([x[2] for x in inputs['run_1'][:-1]],12)

array(['trial_type', 'trial_type', 'trial_type', 'trial_type',
       'trial_type', 'trial_type', 'trial_type', 'trial_type',
       'trial_type', 'trial_type', 'trial_type', 'trial_type', 'blank',
       'blank', 'blank', 'blank', 'blank', 'blank', 'blank', 'blank',
       'blank', 'blank', 'blank', 'blank', 'scrambled', 'scrambled',
       'scrambled', 'scrambled', 'scrambled', 'scrambled', 'scrambled',
       'scrambled', 'scrambled', 'scrambled', 'scrambled', 'scrambled',
       'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy',
       'happy', 'happy', 'happy', 'happy', 'happy', 'sad', 'sad', 'sad',
       'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad',
       'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry',
       'angry', 'angry', 'angry', 'angry', 'angry', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral', 'happy', 'happy',
       'happy', 'hap

In [11]:
#[3dim, label]. have t briefly to match with label, or just have x and y be sorted
#shape=(185*n, *3dim)
#list(data.values())
#something = [list(x.values()) for x in list(fmri_data.values())]
#len(something[0])

In [12]:
print(fmri_data['sub_1']['1'].shape)
print(np.transpose(fmri_data['sub_1']['1']).shape)

(64, 64, 35, 185)
(185, 35, 64, 64)


# Train Test Split

## FULL DATA 

In [15]:
X_train = np.zeros(shape=(64,64,35,185*4)) #X_train takes first 4 runs from all
for i in range(1,17):
  for j in range(1,5):
    X_train[:,:,:,185*(j-1):185*(j)] = fmri_data['sub_'+str(i)][str(j)]
    
X_train = np.transpose(X_train)

y_train = []
for j in range(1,5):
  data = np.repeat([x[2] for x in inputs['run_'+str(j)][1:-1]], 15)[:185]
  for item in data:
    y_train.append(item)


y_train = np.array(y_train)

X_test = np.transpose(fmri_data['sub_1']['5'])
y_test = np.repeat([x[2] for x in inputs['run_5'][1:-1]], 15)[:185]

print(y_train.shape)

(740,)


In [16]:
#CHECK SHAPES
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(740, 35, 64, 64)
(185, 35, 64, 64)
(740,)
(185,)


# Processing

In [17]:
#Turn emotions into specific numbers
#indexing for y. 
emotion = ['blank', 'scrambled','happy','sad','angry','neutral']
y_train_numbered = y_train
y_test_numbered = y_test
for i in range(len(y_train)):
  y_train_numbered[i] = emotion.index(y_train[i])
for i in range(len(y_test)):
  y_test_numbered[i] = emotion.index(y_test[i])

print(y_train_numbered)


['0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '1' '1' '1'
 '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '2' '2' '2' '2' '2' '2'
 '2' '2' '2' '2' '2' '2' '2' '2' '2' '3' '3' '3' '3' '3' '3' '3' '3' '3'
 '3' '3' '3' '3' '3' '3' '4' '4' '4' '4' '4' '4' '4' '4' '4' '4' '4' '4'
 '4' '4' '4' '5' '5' '5' '5' '5' '5' '5' '5' '5' '5' '5' '5' '5' '5' '5'
 '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '3' '3' '3'
 '3' '3' '3' '3' '3' '3' '3' '3' '3' '3' '3' '3' '4' '4' '4' '4' '4' '4'
 '4' '4' '4' '4' '4' '4' '4' '4' '4' '5' '5' '5' '5' '5' '5' '5' '5' '5'
 '5' '5' '5' '5' '5' '5' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1'
 '1' '1' '1' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0'
 '0' '0' '0' '0' '0' '3' '3' '3' '3' '3' '3' '3' '3' '3' '3' '3' '3' '3'
 '3' '3' '5' '5' '5' '5' '5' '5' '5' '5' '5' '5' '5' '5' '5' '5' '5' '0'
 '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '4' '4' '4' '4'
 '4' '4' '4' '4' '4' '4' '4' '4' '4' '4' '4' '1' '1

In [18]:
#Some visualization. of fMRI volumes

# import matplotlib.pyplot as plt
# X_train.shape
# t = 37

# f, axarr = plt.subplots(1,5, figsize=(11, 11)) 
# axarr[0].imshow(X_train[t,:,:,7])
# axarr[1].imshow(X_train[t,:,:,14])
# axarr[2].imshow(X_train[t,:,:,21])
# axarr[3].imshow(X_train[t,:,:,28])
# axarr[4].imshow(X_train[t,:,:,34])

# print('emotion: ',emotion[int(y_train[t])])
# print('t: ',t)

In [19]:
#Idea of our attempt at data augmentation. 

# from torchvision import transforms

# # Define data augmentation transforms
# data_transforms = transforms.Compose([
#     transforms.RandomRotation(10),
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor()
# ])

# # Apply data augmentation during dataset creation
# train_dataset = YourDataset(train_data, transform=data_transforms)

# # Use separate transforms for validation and testing
# val_transforms = transforms.ToTensor()
# test_transforms = transforms.ToTensor()
# val_dataset = YourDataset(val_data, transform=val_transforms)
# test_dataset = YourDataset(test_data, transform=test_transforms)




# PyTorch model

In [20]:
num_epochs = 10
lr = 1e-4
dropout_rate = 0.5

In [21]:
#z-score test
x_mean = np.mean(X_train)
x_std = np.std(X_train)

In [22]:
X_tr_centered = (X_train - x_mean)/x_std
X_ts_centered = (X_test - x_mean)/x_std

In [23]:
X_train_5d = X_train.reshape((X_train.shape[0],1, X_train.shape[1],X_train.shape[2],X_train.shape[3]))
X_test_5d = X_test.reshape((X_test.shape[0],1, X_test.shape[1],X_test.shape[2],X_test.shape[3]))
X_train_5d = torch.from_numpy(X_train_5d).type(torch.float)
X_test_5d = torch.from_numpy(X_test_5d).type(torch.float)
y_train_torch = torch.from_numpy(y_train.astype(int))
y_test_torch = torch.from_numpy(y_test.astype(int))

In [24]:
#CNN MODEL
class Conv3DNet(nn.Module):
    def __init__(self):
        super(Conv3DNet, self).__init__()
        self.conv1 = nn.Conv3d(1, 8, kernel_size=3, padding=1)
        self.conv2 = nn.Conv3d(8, 16, kernel_size=3, padding=1)
        self.conv3 = nn.Conv3d(16, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool3d(kernel_size=2, stride=2)
        #self.fc1 = nn.Linear(32 * 6 * 6 * 6, 512)
        self.fc1 = nn.Linear(8192, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        #x = x.view(-1, 32 * 6 * 6 * 6)
        x = x.view(-1, 8192)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [25]:
#TESTING MORE NEURONS & ONE EXTRA LAYER. NO DIFFERENCE IN ACCURACY

# class Conv3DNet(nn.Module):
#     def __init__(self):
#         super(Conv3DNet, self).__init__()
#         self.conv1 = nn.Conv3d(1, 8, kernel_size=3, padding=2)
#         self.bn1 = nn.BatchNorm3d(8)
#         self.conv2 = nn.Conv3d(8, 16, kernel_size=3, padding=2)
#         self.bn2 = nn.BatchNorm3d(16)
#         self.conv3 = nn.Conv3d(16, 32, kernel_size=3, padding=2)
#         self.bn3 = nn.BatchNorm3d(32)
#         self.conv4 = nn.Conv3d(32, 128, kernel_size=3, padding=2)
#         self.pool = nn.MaxPool3d(kernel_size=2, stride=2)
#         #self.fc1 = nn.Linear(32 * 6 * 6 * 6, 512)
#         self.fc1 = nn.Linear(12800, 512)
#         self.fc2 = nn.Linear(512, 10)
#         self.sig = nn.Sigmoid()
#     def forward(self, x):
#         x = self.pool(torch.sigmoid(self.conv1(x)))
#         x = self.pool(torch.sigmoid(self.conv2(x)))
#         x = self.pool(torch.sigmoid(self.conv3(x)))
#         x = self.pool(torch.sigmoid(self.conv4(x)))
#         #x = x.view(-1, 32 * 6 * 6 * 6)
#         x = x.view(-1, 12800)
#         x = self.sig(self.fc1(x))
#         x = self.fc2(x)
#         return x

In [ ]:
# Instantiate the model, optimizer, and loss function
model = Conv3DNet()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Train the model for num_epoch epochs
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(zip(X_train_5d, y_train_torch)):
        optimizer.zero_grad()
        outputs = model(inputs.unsqueeze(0))
        loss = criterion(outputs, labels.unsqueeze(0).long())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print('Epoch [%d], loss: %.4f' % (epoch+1, running_loss / len(X_train_5d)))

# Evaluate the model on the validation set
# model.eval()
# correct = 0
# total = 0
# with torch.no_grad():
#     for i, (inputs, labels) in enumerate(zip(X_test_5d, y_test_torch)):
#         outputs = model(inputs.unsqueeze(0))
#         _, predicted = torch.max(outputs.data, 1)
#         total += 1
#         correct += (predicted == labels.unsqueeze(0)).sum().item()
# print('Accuracy of the network on the %d validation images: %d %%' % (len(X_test_5d), 100 * correct / total))

Epoch [1], loss: 42.8486
Epoch [2], loss: 1.8355
Epoch [3], loss: 2.1686
Epoch [4], loss: 1.8284
Epoch [5], loss: 1.8212
Epoch [6], loss: 1.8189
Epoch [7], loss: 1.8169
Epoch [8], loss: 1.8144
Epoch [9], loss: 1.8127


In [ ]:
#Accuracy computations
def compute_accuracy(x,y):
  correct = 0

  total = 0
  with torch.no_grad():
      for i, (inputs, labels) in enumerate(zip(x, y)):
          outputs = model(inputs.unsqueeze(0))
          _, predicted = torch.max(outputs.data, 1)
          total += 1
          correct += (predicted == labels.unsqueeze(0)).sum().item()
  print('Accuracy on %d images: %d %%' % (len(x), 100 * correct / total))

print("train acc: ")
compute_accuracy(X_train_5d, y_train_torch)
print("test acc: ")
compute_accuracy(X_test_5d, y_test_torch)